In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy

/Users/ibiscp/GitHub/logspace/wavy/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [5]:
hist.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,19.040893,19.147466,18.934321,18.998264,65002900,0.0,0
2005-01-04,19.090624,19.254034,18.941422,19.069309,109442100,0.0,0
2005-01-05,19.069310,19.254036,19.012472,19.026682,72463500,0.0,0
2005-01-06,19.076421,19.225622,18.927220,19.005373,76890500,0.0,0
2005-01-07,19.055104,19.104837,18.913009,18.948532,68723300,0.0,0
2005-01-10,18.898798,19.083523,18.856170,19.040894,70376600,0.0,0
2005-01-11,18.962747,19.055109,18.905908,18.991165,64712000,0.0,0
2005-01-12,19.019577,19.076415,18.913005,19.026682,72940600,0.0,0
2005-01-13,18.955638,19.040895,18.586188,18.664341,89861600,0.0,0


In [6]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [7]:
hist.Dividends.plot()

In [8]:
hist['Stock Splits'].plot()

In [9]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [10]:
hist = hist.pct_change()

In [11]:
hist.Close.plot()

In [12]:
print(len(hist) - len(hist.dropna()))
hist.dropna(inplace=True)

1


In [13]:
hist.High.hist(bins=200)

In [14]:
x, y = wavy.create_panels(hist, lookback=10, horizon=1)

In [16]:
x.plot()

In [15]:
y[0]

,Open,High,Low,Close
Date,,,,
2005-01-19,0.006915,-0.003416,0.003096,-0.012918


In [17]:
# predict only the highest price
y = y[['High']]

In [18]:
# try single col inputs first
x_high = x[['High']]

In [19]:
# constant = wavy.ConstantModel(x_high, y, model_type="regression")
# constant.fit()

In [29]:
baseline = wavy.BaselineModel(x_high, y, model_type="regression")
baseline.fit() # val_MAE: 0.0136

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

In [ ]:
lr = wavy.LinearRegression(x_high, y)
lr.fit(epochs=20)

In [ ]:
y.test[0]

In [ ]:
y.test[1]

In [ ]:
baseline.x_test.shape

In [ ]:
from copy import deepcopy

def update(panel, other):
    panel = deepcopy(panel)

    for i, j in zip(panel, other):
        i.iloc[:,:] = j
    return panel

In [ ]:
y.test.shape

In [ ]:
a = update(y.test[1:], baseline.predict())

In [ ]:
a[0]

In [ ]:
y.test[1]

In [ ]:
copy(y.test)

In [ ]:
b = baseline.predict()

In [ ]:
b[0]

In [ ]:
dense = wavy.DenseModel(x_high, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression")
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)